In [1]:
# Import Libraries
import pickle

import mlflow
import pandas as pd
import xgboost
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from mlflow.tracking import MlflowClient
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error


In [2]:
# defining a function to quickly read and prepare data
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['hour'] = df.lpep_pickup_datetime.dt.hour
    df['dayofweek'] = df.lpep_pickup_datetime.dt.day_of_week

    categorical = ['PULocationID', 'DOLocationID', 'hour', 'dayofweek', 'VendorID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [3]:
# Reading and preparing the dataset
df_train = read_dataframe('/home/ubuntu/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/ubuntu/data/green_tripdata_2021-02.parquet')

In [4]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment-xgboost")

<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment-xgboost', tags={}>

In [5]:
# defing feature types
categorical = ['PU_DO', 'hour', 'dayofweek', 'VendorID'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# applying one hot encoding
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

with open("models/dvtranformer.bin", 'wb') as fileout:
    pickle.dump(dv, fileout)

In [6]:
# defining the target
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
# defining xboost level data

training_data = xgboost.DMatrix(X_train, label=y_train)
valid = xgboost.DMatrix(X_val, label=y_val)

In [8]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        model = xgboost.train(
            params = params,
            dtrain = training_data,
            num_boost_round = 100,
            evals = [(valid, "validation")],
            early_stopping_rounds = 20
        )

        y_pred = model.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -6, 1),
    'reg_lambda': hp.loguniform('reg_lambda', -5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective' : 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)



[0]	validation-rmse:14.41132                          
[1]	validation-rmse:10.36988                          
[2]	validation-rmse:8.16466                           
[3]	validation-rmse:7.08518                           
[4]	validation-rmse:6.60465                           
[5]	validation-rmse:6.39788                           
[6]	validation-rmse:6.31192                           
[7]	validation-rmse:6.27025                           
[8]	validation-rmse:6.24479                           
[9]	validation-rmse:6.23351                           
[10]	validation-rmse:6.22605                          
[11]	validation-rmse:6.21779                          
[12]	validation-rmse:6.21442                          
[13]	validation-rmse:6.20850                          
[14]	validation-rmse:6.20372                          
[15]	validation-rmse:6.20001                          
[16]	validation-rmse:6.19793                          
[17]	validation-rmse:6.19466                          
[18]	valid

In [9]:
# Train the best model

mlflow.xgboost.autolog(disable=True)

best_result['max_depth'] = int(best_result['max_depth'])
best_result['objective'] = 'reg:squarederror'
best_result['seed'] = 42

#mlflow.xgboost.autolog()
with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(best_result)
    model = xgboost.train(
            params = best_result,
            dtrain = training_data,
            num_boost_round = 200,
            evals = [(valid, "validation")],
            early_stopping_rounds = 50
        )
    y_pred = model.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/dvtranformer.bin",
                        artifact_path="transformer/dvtranformer.bin")
    mlflow.xgboost.log_model(xgb_model=model,artifact_path="models_mlflow")
            


[0]	validation-rmse:14.41132
[1]	validation-rmse:10.36988
[2]	validation-rmse:8.16466
[3]	validation-rmse:7.08518
[4]	validation-rmse:6.60465
[5]	validation-rmse:6.39788
[6]	validation-rmse:6.31192
[7]	validation-rmse:6.27025
[8]	validation-rmse:6.24479
[9]	validation-rmse:6.23351
[10]	validation-rmse:6.22605
[11]	validation-rmse:6.21779
[12]	validation-rmse:6.21442
[13]	validation-rmse:6.20850
[14]	validation-rmse:6.20372
[15]	validation-rmse:6.20001
[16]	validation-rmse:6.19793
[17]	validation-rmse:6.19466
[18]	validation-rmse:6.19239
[19]	validation-rmse:6.19080
[20]	validation-rmse:6.18895
[21]	validation-rmse:6.18751
[22]	validation-rmse:6.18436
[23]	validation-rmse:6.18323
[24]	validation-rmse:6.18161
[25]	validation-rmse:6.17993
[26]	validation-rmse:6.17837
[27]	validation-rmse:6.17716
[28]	validation-rmse:6.17589
[29]	validation-rmse:6.17450
[30]	validation-rmse:6.17265
[31]	validation-rmse:6.17184
[32]	validation-rmse:6.17105
[33]	validation-rmse:6.17011
[34]	validation-rmse:6

/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [10]:
#Load the model and artifact
run_id = 'bbc9d7ab033d407cae47760482721b6c'
model_uri = f'runs:/{run_id}/models_mlflow'
loaded_model = mlflow.pyfunc.load_model(model_uri)

print(loaded_model)


y_pred = loaded_model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(rmse)

# load dict vectoriser
client = MlflowClient()
client.download_artifacts(run_id, "transformer", "models")

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: bbc9d7ab033d407cae47760482721b6c

6.117274781508819


'/home/ubuntu/mlops-zoomcamp/02-experiment-tracking/models/transformer'